<a href="https://colab.research.google.com/github/SohaHussain/Glaucoma-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Dependencies

In [ ]:
# for linear algebra
import numpy as np

# for data processing and CSV files I/O
import pandas as pd

# for file handling
import os

import pickle
import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as k
from google.colab import files
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model 

In [ ]:
print(np.__version__)
print(pd.__version__)


print(keras.__version__)


1.21.6
1.3.5
2.8.0


##Organizing Data

In [ ]:
# train directory
train_dir = '/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Train/'

# validation directory
val_dir = '/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Validation/'

In [ ]:
# number of negative images in train directory
list = os.listdir('/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Train/Glaucoma_Negative')
number_files = len(list)
print(number_files)

386


In [ ]:
# number of positive images in train directory
list = os.listdir('/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Train/Glaucoma_Positive')
number_files = len(list)
print(number_files)

134


In [ ]:
# number of positive images in validation directory
list = os.listdir('/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Validation/Glaucoma_Positive')
number_files = len(list)
print(number_files)

34


In [ ]:
# number of negative images in validation directory
list = os.listdir('/content/drive/MyDrive/health app datasets/Train_Val_Data/scans_sorted/Validation/Glaucoma_Negative')
number_files = len(list)
print(number_files)

96


## Model variables

In [ ]:
img_width = 150
img_height = 150
nb_train_samples = 520
nb_validation_samples = 130
epochs = 10
batch_size = 16

In [ ]:
if k.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


##Building a Neural Network

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

##Data Augmentation

In [ ]:
# this is the augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 520 images belonging to 2 classes.
Found 130 images belonging to 2 classes.


##Training the model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


Epoch 1/10
32/32 [==============================] - 247s 7s/step - loss: 0.6711 - accuracy: 0.7163 - val_loss: 0.5905 - val_accuracy: 0.7422
Epoch 2/10
32/32 [==============================] - 46s 1s/step - loss: 0.5862 - accuracy: 0.7421 - val_loss: 0.5840 - val_accuracy: 0.7344
Epoch 3/10
32/32 [==============================] - 46s 1s/step - loss: 0.5991 - accuracy: 0.7381 - val_loss: 0.5767 - val_accuracy: 0.7344
Epoch 4/10
32/32 [==============================] - 48s 2s/step - loss: 0.6042 - accuracy: 0.7361 - val_loss: 0.5770 - val_accuracy: 0.7344
Epoch 5/10
32/32 [==============================] - 46s 1s/step - loss: 0.5990 - accuracy: 0.7440 - val_loss: 0.5970 - val_accuracy: 0.7422
Epoch 6/10
32/32 [==============================] - 46s 1s/step - loss: 0.5934 - accuracy: 0.7401 - val_loss: 0.5990 - val_accuracy: 0.7344
Epoch 7/10
32/32 [==============================] - 46s 1s/step - loss: 0.5879 - accuracy: 0.7401 - val_loss: 0.5991 - val_accuracy: 0.7344
Epoch 8/10
32/32 [=

##Testing the model

In [ ]:
upload = files.upload()
for fn in upload.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=batch_size)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " has glaucoma")
  else:
    print(fn + " does not have glaucoma")

##Saving the model

In [ ]:
model.save('/content/drive/My Drive/folder(1)/my_model.h5')



In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/folder(1)/ImageClassifier.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/folder(1)/ImageClassifier.h5")